# Convert .mat file to hdf5 python format

HDF5 describes data set dimensions in **row-major order**; MATLAB stores data in **column-major order**.For example, if Matlab (Fortran) writes a 4x6 two-dimensional dataset to the file, a Python (C) program will read it as a 6x4 two-dimensional dataset into memory.

"The first dimension stored in the list of dimensions is the slowest changing dimension and the last dimension stored is the fastest changing dimension."

scipy.io.loadmat only soppourt version less than 7.3. To read version 7.3 we need hdf5.   

In Matlab you can put the flag '-v7.3' to save as hdh5 file.   
save('filename', '-v7.3', 'var1');

More about matlab version [here](https://www.mathworks.com/help/matlab/import_export/mat-file-versions.html)

In [2]:
import sys

import h5py
import matplotlib.pyplot as plt
from scipy.io import loadmat

sys.path.append('../src')
from mealib.stimulus import correct_checkerboard
from mealib.utils import check_directory

%matplotlib notebook


inputs:   
* **exp_name** : name of experiment
* **checkerboar_file**: file with only checkerboar syncronization time. To generate this file use Get-sync notebook
* **stim_file** : stim im .mat format
* **sync_file** : full path to checkerboar_file
* **repeated_file** : txt file with repeated frame
* **outputfile** : full path to save converted stimulus


In [ ]:
exp_name = 'MR-0242'
checkerboar_file = '023.txt'

stim_file = '../data/stim/checkerboard/' + exp_name + '.mat'    
sync_file = '../data/sync/' + exp_name + '/event_list/' + checkerboar_file
repeated_file  =  '../data/sync/' + exp_name + '/repeated_frames_' + exp_name + '.txt'
outputfolder = '../data/stim/checkerboard/'
check_directory(outputfolder)
outputfile = outputfolder+'stim_mini_' + exp_name + '_.hdf5'

In [ ]:
correct_checkerboard(sync_file, repeated_file, outputfile, stim_file)

# Check the correct convertion 

In [ ]:
# Load First image from log file
first_img_file = '../data/raw_data/MR-0227/log/Exp__2018_01_25-11.21.37-Whitenoise_35x35_20min/FirstImages_1.mat'
first_img = loadmat(first_img_file)['fi']
print(first_img.dtype,first_img.shape)

In [ ]:
# Load corrected stim file
stim = h5py.File(outputfile,'r')
print(stim['checkerboard'].dtype, stim['checkerboard'].shape)

In [ ]:
idx_img = 0

plt.figure()
plt.subplot(1,2,1).set_title('Matlab first image (pixels)')
plt.imshow(first_img[...,idx_img])
plt.subplot(1,2,2).set_title('hdf5 first image (blocks)')
plt.imshow(np.transpose(stim['checkerboard'][idx_img,...],(1,2,0)))
# plt.savefig('../figures/first_image_checkerboar.png')

In [ ]:
plt.figure()
plt.subplot(1,2,1).set_title('Matlab first image')
plt.pcolor(stim['checkerboard'][13951,1,...])
plt.subplot(1,2,2).set_title('hdf5 first image')
plt.pcolor(stim['checkerboard'][13952,1,...])
# plt.savefig('../figures/first_imag28232e_checkerboar.png')

In [ ]:
stim.close()